## Leap Ahead with Redis 6.2

## Brian Sam-Bodden

- Developer Advocate at Redis
- @bsbodden on (Twitter|GitHub|GitLab|LinkedIn)
- Technologist
- Entrepreneur
- Author
- Teacher, and Student
- Aims to push the envelope of technology and fill the world with better, more passionate programmers.
- Brings vision and energy, and willingness to start at the beginning.

## DaShaun Carter

- 319 days as Partner Solution Architect, Redis
- @dashaun on (Twitter|GitHub|GitLab|LinkedIn)
- A husband, father of four, volunteer and struggling athlete.
- Deliberately practicing to build better software, faster.
- Generalist.
- Computer Science. MBA.
- Continuously learning.
- Believes that every rep counts.
- Doesn't have anything figured out.
- Trying to get a little better every day.
- Understanding in order to be understood.

### Redis

- The "Most Loved" database in StackOverflow's Developer Survey
- For the 5th year in a row
- Data stored in memory, not on disk
- < 1m latency
- 162 clients in 50 different languages

### Redis OSS 6.2
#### The Community Edition

- 6.2 was driven by the community
- Loads of community contributed features
- 15 new commands
- 5 commands changed
- 8 commands deprecated

### Spring Data

### Spring Data Redis

#### String : ValueOperations

SET -> set

In [ ]:
redis-cli set leapaheadkey leapaheadvalue

```java
@Component
class Set implements Function<KeyValue, String> {
	private final StringRedisTemplate redisTemplate;
	public ValueSet(StringRedisTemplate redisTemplate){
		this.redisTemplate = redisTemplate;
	}
	@Override
	public String apply(KeyValue input) {
		redisTemplate.opsForValue().set(input.getKey(), input.getValue());
		return "OK";
	}
}
```

In [ ]:
curl -H "Content-Type: application/json" localhost:8080/set -d '{"key":"leapahead-string","value":"Spring Cloud Function"}'

### String

GET -> get

In [ ]:
redis-cli get leapaheadkey

```java
@Component
class Get implements Function<String, String> {
	private final StringRedisTemplate redisTemplate;
	public ValueGet(StringRedisTemplate redisTemplate){
		this.redisTemplate = redisTemplate;
	}
	@Override
	public String apply(String input) {
		return redisTemplate.opsForValue().get(input);
	}
}
```

In [ ]:
curl -H "Content-Type: text/plain" localhost:8080/get -d 'leapaheadkey'

#### String : ValueOperations

GETSET -> getAndSet

In [ ]:
redis-cli set leapaheadkey leapaheadvalue
redis-cli get leapaheadkey
redis-cli getset leapaheadkey updatedvalue
redis-cli getset leapaheadkey anotherupdatedvalue
redis-cli get leapaheadkey

```java
@Component
class GetAndSet implements Function<KeyValue, String> {
	private final StringRedisTemplate redisTemplate;
	public GetAndSet(StringRedisTemplate redisTemplate){
		this.redisTemplate = redisTemplate;
	}
	@Override
	public String apply(KeyValue input) {
		return redisTemplate.opsForValue().getAndSet(input.getKey(), input.getValue());
	}
}
```

In [ ]:
curl -H "Content-Type: application/json" localhost:8080/set -d '{"key":"leapahead-string","value":"Spring Cloud Function"}'
echo ''
curl -H "Content-Type: application/json" localhost:8080/getAndSet -d '{"key":"leapahead-string","value":"This is so cool!"}'
echo ''
curl -H "Content-Type: text/plain" localhost:8080/get -d 'leapahead-string'

#### String : ValueOperations

GETEX -> getAndExpire

In [8]:
redis-cli set leapaheadkey 'This message should self destruct in 3 seconds'
redis-cli getex leapaheadkey ex 3
sleep 1
redis-cli ttl leapaheadkey
sleep 2
redis-cli get leapaheadkey

OK
"This message should self destruct in 3 seconds"
(integer) 2
(nil)


```java
@Component
class GetEx implements Function<KeyValue, String> {
	private final StringRedisTemplate redisTemplate;
	public GetEx(StringRedisTemplate redisTemplate){
		this.redisTemplate = redisTemplate;
	}
	@Override
	public String apply(KeyValue input) {
		Duration t = Duration.ofSeconds(Long.parseLong(input.getValue()));
		return redisTemplate
				.opsForValue()
				.getAndExpire(input.getKey(), t);
	}
}
```

In [4]:
curl -H "Content-Type: application/json" localhost:8080/set -d '{"key":"leapahead-string","value":"Self-Destructing"}'
echo ''
curl -H "Content-Type: application/json" localhost:8080/getEx -d '{"key":"leapahead-string","value":"3"}'
echo ''
sleep 3
curl -H "Content-Type: text/plain" localhost:8080/get -d 'leapahead-string'

OK
Self-Destructing
{"timestamp":"2021-09-01T22:31:33.074+00:00","path":"/get","status":500,"error":"Internal Server Error","requestId":"f90f5244-1"}

#### String : ValueOperations

GETDEL -> getAndDelete

In [5]:
redis-cli set leapaheadkey 'One Per Customer'
redis-cli getdel leapaheadkey
redis-cli get leapaheadkey

OK
"One Per Customer"
(nil)


```java
@Component
class GetDel implements Function<String, String> {
	private final StringRedisTemplate redisTemplate;
	public GetDel(StringRedisTemplate redisTemplate){
		this.redisTemplate = redisTemplate;
	}
	@Override
	public String apply(String input) {
		return redisTemplate.opsForValue().getAndDelete(input);
	}
}
```

In [9]:
curl -H "Content-Type: application/json" localhost:8080/set -d '{"key":"leapahead-string","value":"Self-Destructing"}'
echo ''
curl -H "Content-Type: application/json" localhost:8080/getDel -d 'leapahead-string'
echo ''
curl -H "Content-Type: text/plain" localhost:8080/get -d 'leapahead-string'

OK
Self-Destructing
{"timestamp":"2021-09-01T22:41:57.251+00:00","path":"/get","status":500,"error":"Internal Server Error","requestId":"7eb79a20-1"}

#### String : ValueOperations

SET PX & EX

In [5]:
redis-cli set leapaheadkey 'One Per Customer'
redis-cli getdel leapaheadkey
redis-cli get leapaheadkey

OK
"One Per Customer"
(nil)


```java
@Component
class GetDel implements Function<String, String> {
	private final StringRedisTemplate redisTemplate;
	public GetDel(StringRedisTemplate redisTemplate){
		this.redisTemplate = redisTemplate;
	}
	@Override
	public String apply(String input) {
		return redisTemplate.opsForValue().getAndDelete(input);
	}
}
```

In [9]:
curl -H "Content-Type: application/json" localhost:8080/set -d '{"key":"leapahead-string","value":"Self-Destructing"}'
echo ''
curl -H "Content-Type: application/json" localhost:8080/getDel -d 'leapahead-string'
echo ''
curl -H "Content-Type: text/plain" localhost:8080/get -d 'leapahead-string'

OK
Self-Destructing
{"timestamp":"2021-09-01T22:41:57.251+00:00","path":"/get","status":500,"error":"Internal Server Error","requestId":"7eb79a20-1"}

#### String : ValueOperations

SET PXAT & EXAT

In [12]:
redis-cli set leapaheadkey 'This offer will not last' PXAT 1662163200000
echo 'There are 31536000 seconds in a year'
redis-cli ttl leapaheadkey
redis-cli set leapaheadkey2 'I will glady pay you later for a hamburger today' EXAT 1662163200
redis-cli ttl leapaheadkey2

OK
There are 31536000 seconds in a year
(integer) 31610130
OK
(integer) 31610130


```java
@Component
class SetPX implements Function<PX, String> {
	private final StringRedisTemplate redisTemplate;
	public SetPX(StringRedisTemplate redisTemplate){
		this.redisTemplate = redisTemplate;
	}
	@Override
	public String apply(PX input) {
		Objects.requireNonNull(redisTemplate.getConnectionFactory())
				.getConnection()
				.set(input.getKey().getBytes(StandardCharsets.UTF_8),
						input.getValue().getBytes(StandardCharsets.UTF_8),
						Expiration.unixTimestamp(Long.parseLong(input.getP()), TimeUnit.MILLISECONDS),
						RedisStringCommands.SetOption.UPSERT);
		return "OK";
	}
}
```

In [30]:
curl -H "Content-Type: application/json" localhost:8080/setPx -d '{"key":"leapahead-string","value":"Self-Destructing","p":"'"$(($(date +%s000) + 2000))"'"}'
echo ''
curl -H "Content-Type: text/plain" localhost:8080/get -d 'leapahead-string'
sleep 2
echo ''
curl -H "Content-Type: text/plain" localhost:8080/get -d 'leapahead-string'



OK
Self-Destructing
{"timestamp":"2021-09-02T03:51:59.583+00:00","path":"/get","status":500,"error":"Internal Server Error","requestId":"42757f7d-1"}